# 通过 PyTorch 进行深度学习的简介

在此 notebook 中，你将了解 [PyTorch](http://pytorch.org/)，一款用于构建和训练神经网络的框架。PyTorch 在很多方面的行为都和你喜欢的 Numpy 数组很像。这些 Numpy 数组毕竟只是张量。PyTorch 采用这些张量并使我们能够轻松地将张量移到 GPU 中，以便在训练神经网络时加快处理速度。它还提供了一个自动计算梯度（用于反向传播！）的模块，以及另一个专门用于构建神经网络的模块。总之，与 TensorFlow 和其他框架相比，PyTorch 与 Python 和 Numpy/Scipy 堆栈更协调。



## 神经网络

深度学习以人工神经网络为基础，而后者从上世纪 50 年代末就出现了。神经网络由像神经元一样的单个部分组成，这些部分通常称为单元或直接叫做“神经元”。每个单元都具有一定数量的加权输入。我们对这些加权输入求和（线性组合），然后将结果传递给激活函数，以获得单元的输出 。

<img src="assets/simple_neuron.png" width=400px>

数学公式如下所示：

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i \right)
\end{align}
$$

对于向量来说，为两个向量的点积/内积：

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

### 堆叠起来！

我们可以将这些单元神经元组合为层和堆栈，形成神经元网络。一个神经元层的输出变成另一层的输入。对于多个输入单元和输出单元，我们现在需要将权重表示为矩阵。

<img src='assets/multilayer_diagram_weights.png' width=450px>

我们再次能够用矩阵以数学方式表示这些数据，并使用矩阵乘法获得一次运算中每个单元的线性组合。例如，隐藏层（此以下公式中为 $h_1$ 和 $h_2$）可以计算为 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

我们通过将隐藏层当做输出单元的输入，可以算出这个小型网络的输出。网络输出简单地表示为

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

## 张量

实际上神经网络计算只是对张量进行一系列线性代数运算，矩阵是张量的一种形式。向量是一维张量，矩阵是二维张量，包含 3 个索引的数组是三维张量（例如 RGB 颜色图像）。神经网络的基本数据结构是张量，PyTorch（以及几乎所有其他深度学习框架）都是以张量为基础。

<img src="assets/tensor_examples.svg" width=600px>

介绍了基本知识后，现在该了解如何使用 PyTorch 构建简单的神经网络了。

In [1]:
!pip install helper

  Preparing metadata (setup.py) ... done
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19169 sha256=633dac2b1cf45d4f2e7a9268a6461a662b458671ab157eccaf283a71925b239b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/13/8a/92/4a4267b8371d6e453121c917021173b4a682d691f123b9f647
Successfully built helper


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

IOStream.flush timed out


首先，我们来看看如何处理 PyTorch 张量。这些结构是神经网络和 PyTorch 的基本数据结构，因此请务必理解这些运算的原理。

In [3]:
x = torch.rand(3, 2)
x

tensor([[0.9785, 0.8465],
        [0.3841, 0.3027],
        [0.2568, 0.0955]])

In [4]:
y = torch.ones(x.size())
y

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [5]:
z = x + y
z

tensor([[1.9785, 1.8465],
        [1.3841, 1.3027],
        [1.2568, 1.0955]])

一般而言，PyTorch 张量的行为和 Numpy 数组相似。它们的索引都以 0 开始，并且支持切片。

In [6]:
z[0]

tensor([1.9785, 1.8465])

In [7]:
z[:, 1:]

tensor([[1.8465],
        [1.3027],
        [1.0955]])

张量通常有两种类型的方法，一种方法返回另一个张量，另一种方法原地执行运算。即该张量在内存中的值发生了改变，没有创建新的张量。原地函数始终带有下划线，例如 `z.add()` 和 `z.add_()`。

In [8]:
# Return a new tensor z + 1
z.add(1)

tensor([[2.9785, 2.8465],
        [2.3841, 2.3027],
        [2.2568, 2.0955]])

In [9]:
# z tensor is unchanged
z

tensor([[1.9785, 1.8465],
        [1.3841, 1.3027],
        [1.2568, 1.0955]])

In [10]:
# Add 1 and update z tensor in-place
z.add_(1)

tensor([[2.9785, 2.8465],
        [2.3841, 2.3027],
        [2.2568, 2.0955]])

In [11]:
# z has been updated
z

tensor([[2.9785, 2.8465],
        [2.3841, 2.3027],
        [2.2568, 2.0955]])

### 改变形状

改变张量的形状是一个很常见的运算。首先使用 `.size()`获取张量的大小和形状。然后，使用 `.resize_()`改变张量的形状。注意下划线，改变形状是原地运算。

In [12]:
z.size()

torch.Size([3, 2])

In [13]:
z.resize_(2, 3)

tensor([[2.9785, 2.8465, 2.3841],
        [2.3027, 2.2568, 2.0955]])

In [14]:
z

tensor([[2.9785, 2.8465, 2.3841],
        [2.3027, 2.2568, 2.0955]])

## 在 Numpy 与 Torch 之间转换

在 Numpy 数组与 Torch 张量之间转换非常简单并且很实用。要通过 Numpy 数组创建张量，使用 `torch.from_numpy()`。要将张量转换为 Numpy 数组，使用 `.numpy()` 方法。

In [15]:
a = np.random.rand(4,3)
a

array([[0.26687869, 0.47968318, 0.65862941],
       [0.60571515, 0.31351613, 0.78648828],
       [0.87265537, 0.90423262, 0.81094658],
       [0.12350148, 0.45721362, 0.94592636]])

In [16]:
b = torch.from_numpy(a)
b

tensor([[0.2669, 0.4797, 0.6586],
        [0.6057, 0.3135, 0.7865],
        [0.8727, 0.9042, 0.8109],
        [0.1235, 0.4572, 0.9459]], dtype=torch.float64)

In [17]:
b.numpy()

array([[0.26687869, 0.47968318, 0.65862941],
       [0.60571515, 0.31351613, 0.78648828],
       [0.87265537, 0.90423262, 0.81094658],
       [0.12350148, 0.45721362, 0.94592636]])

内存在 Numpy 数组与 Torch 张量之间共享，因此如果你原地更改一个对象的值，另一个对象的值也会更改。

In [18]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[0.5338, 0.9594, 1.3173],
        [1.2114, 0.6270, 1.5730],
        [1.7453, 1.8085, 1.6219],
        [0.2470, 0.9144, 1.8919]], dtype=torch.float64)

In [19]:
# Numpy array matches new values from Tensor
a

array([[0.53375738, 0.95936636, 1.31725881],
       [1.21143031, 0.62703227, 1.57297656],
       [1.74531074, 1.80846524, 1.62189316],
       [0.24700297, 0.91442724, 1.89185271]])